In [27]:
from langchain.agents import create_agent
from tools import web_search, get_current_weather, calculator
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver
from pydantic import BaseModel, Field
import os
from dotenv import load_dotenv
load_dotenv()
from langgraph.graph import MessagesState
from langgraph.graph import StateGraph, START, END
from typing import Literal
from langgraph.types import Command
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
import json
checkpointer = InMemorySaver()
config = {"configurable": {"thread_id": "1"}}
model = ChatOpenAI(base_url=os.getenv("BASE_URL"), api_key=os.getenv("GROQ_API_KEY"), model=os.getenv("MODEL_NAME", "gpt-4o"))


class State(TypedDict):
    messages: Annotated[list, add_messages]
    next_agent: Literal["search_agent", "calculator_agent", "weather_agent"] = Field(..., description="The next agent to delegate the task to.")   





communicate_agent = create_agent(
    model=model,
    system_prompt="You are a helpful AI assistant. You receive user queries and delegate tasks to specialized agents: search_agent, calculator_agent, and weather_agent. Use the responses from these agents to formulate a final answer to the user. Always use context from Knowledge Base (RAG) and available tools to answer precisely. Answer in short all needed information",
    response_format=State
)
user_query = (
        "Based on the knowledge base, what is the historical average temperature in Da Lat? "
        "Now, use a tool to get the current real-time temperature in Da Lat. "
        "Then, use the calculator to find the difference between the current temperature and the historical average. "
        "Finally, search for one interesting fact about Da Lat's flower festival."
    )
response = communicate_agent.invoke({"messages": user_query})


raw_content = response["messages"][-1].content

parsed = json.loads(raw_content)

content = parsed["messages"][-1]["content"]
next_agent = parsed["next_agent"]

print(content)
print(next_agent)

Based on the knowledge base, what is the historical average temperature in Da Lat? Now, use a tool to get the current real-time temperature in Da Lat. Then, use the calculator to find the difference between the current temperature and the historical average. Finally, search for one interesting fact about Da Lat's flower festival.
weather_agent
